In [22]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from scipy.integrate import solve_ivp
import matplotlib
matplotlib.use('TkAgg')

# ---------------- physical & numerical constants ----------------
g        = 9.81      # gravity (m s⁻²)
L        = 1.0       # pendulum length (m)
A        = 0.01      # vertical drive amplitude (m)
omega_d  = 100.0      # drive angular frequency (rad s⁻¹)

theta0   = np.pi     # initial angle (rad)  – near upright
thetadot0 = 0.05     # initial angular velocity (rad s⁻¹)

T        = 10.0      # total simulated time (s)
fps      = 60        # animation frame rate
frames   = int(T * fps)
t_eval   = np.linspace(0.0, T, frames)

# ---------------- ODE:  y = [θ, θ̇] ----------------
def kapitza_rhs(t, y):
    θ, θ̇ = y
    g_eff = g + A * omega_d**2 * np.cos(omega_d * t)
    return [θ̇, -(g_eff / L) * np.sin(θ)]

sol = solve_ivp(
    kapitza_rhs, (0.0, T), [theta0, thetadot0],
    t_eval=t_eval, rtol=1e-8, atol=1e-9, method="DOP853"
)
theta = sol.y[0]

# ---------------- set up Matplotlib animation ----------------
fig, ax = plt.subplots(figsize=(6, 6))
ax.set_aspect("equal"); ax.axis("off")
ax.set_xlim(-1.2 * L, 1.2 * L)
ax.set_ylim(-1.2 * L, 1.2 * L)

rod, = ax.plot([], [], lw=2)
bob, = ax.plot([], [], "ro", ms=8)

def init():
    rod.set_data([], [])
    bob.set_data([], [])
    return rod, bob

def update(i):
    x =  L * np.sin(theta[i])
    y = -L * np.cos(theta[i])
    rod.set_data([0, x], [0, y])
    bob.set_data([x], [y])
    return rod, bob

ani = FuncAnimation(
    fig, update, frames=frames, init_func=init,
    interval=1000 / fps, blit=True
)

plt.show()        # works in scripts; Jupyter shows the animation inline


In [23]:
# -------------------------------------------------------------------
# Phase-space analysis for the Kapitza pendulum
# (assumes you already executed the integration cell that defines
#  `theta`, `sol.y[1]`, `t_eval`, `omega_d`)
# -------------------------------------------------------------------
import numpy as np
import matplotlib.pyplot as plt

theta      = sol.y[0]          # θ(t)
theta_dot  = sol.y[1]          # θ̇(t)

# ----- helper: wrap angles into (-π, π] so the portrait is compact
def wrap(ang):
    return (ang + np.pi) % (2*np.pi) - np.pi

theta_wrapped = wrap(theta)

# ----- build the Poincaré section (sample once per drive period)
T_drive   = 2*np.pi / omega_d
p_times   = np.arange(0, t_eval[-1], T_drive)         # 0, T_d, 2T_d, …
theta_P   = np.interp(p_times, t_eval, theta)         # linear interp is fine
thetadot_P= np.interp(p_times, t_eval, theta_dot)

# ----- plot -----------------------------------------------------------------
fig, ax = plt.subplots(figsize=(6, 5))

# full trajectory
ax.plot(theta_wrapped, theta_dot, lw=0.8, color="tab:orange", label="trajectory")

# stroboscopic section
ax.scatter(wrap(theta_P), thetadot_P, s=20, color="red", zorder=3,
           label=r"Poincaré ($t = nT_d$)")

ax.set_xlabel(r"$\theta$  [rad]")
ax.set_ylabel(r"$\dot{\theta}$  [rad s$^{-1}$]")
ax.set_title("Phase portrait – Kapitza pendulum")
ax.grid(alpha=0.4)
ax.legend(loc="upper left")

plt.tight_layout()
plt.show()
